In [1]:
import cv2 as cv
from ultralytics import YOLO
from ultralytics.yolo.utils.plotting import Annotator

WARNING ⚠️ 'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING ⚠️ 'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.utils' instead.
Note this warning may be related to loading older models. You can update your model to current structure with:
    import torch
    ckpt = torch.load("model.pt")  # applies to both official and custom models
    torch.save(ckpt, "updated-model.pt")



In [2]:
model = YOLO('../best.pt')

In [3]:
class_names = open('../classes.txt').read().strip().splitlines()
class_names

['window', 'window-outside']

In [4]:
image = cv.imread('../window_example.jpg')
image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
results = model.predict(image, conf=0.5, classes=0)
annotator = Annotator(image)


0: 448x640 2 windows, 92.6ms
Speed: 8.1ms preprocess, 92.6ms inference, 0.8ms postprocess per image at shape (1, 3, 448, 640)


In [10]:
for result in results:
    for box in result.boxes:
        class_id = int(box.cls)
        class_name = class_names[class_id]
        bbox = box.xywh[0]
        bbox_xy = bbox[:2]
        bbox_wh = bbox[:2]
        print(class_name)
        print(bbox_xy, bbox_wh)
        # bbx0, bby0, bbx1, bby1 = map(int, box.xyxy[0])
        # bbxw = bbx1 - bbx0
        # bbyh = bby1 - bby0
        # cv.rectangle(image, (bbx0, bby0), (bbxw, bbyh), (0, 255, 0), thickness=2)

# cv.imshow('image', image)
# cv.waitKey()
# cv.destroyAllWindows()

window
tensor([567.1852, 224.4161]) tensor([567.1852, 224.4161])
window
tensor([973.2778, 224.7985]) tensor([973.2778, 224.7985])
